In [11]:
from pyspark.sql import functions as F
from pyspark.sql.types import DateType, IntegerType
from datetime import datetime

StatementMeta(, 27e95eb9-9622-40eb-9543-3e616371ab29, 13, Finished, Available, Finished)

In [12]:
orders = spark.sql("SELECT * FROM lh_silver.orders")
order_items = spark.sql("SELECT * FROM lh_silver.order_items")
wishlists = spark.sql("SELECT * FROM lh_silver.wishlist")

StatementMeta(, 27e95eb9-9622-40eb-9543-3e616371ab29, 14, Finished, Available, Finished)

In [13]:
min_order_date = orders.select(F.min('order_date')).collect()[0][0]
min_order_items_date = order_items.select(F.min('date')).collect()[0][0]
mid_wishlists_date = wishlists.select(F.min('wishlist_date')).collect()[0][0]

start_date = min(min_order_date, min_order_items_date, mid_wishlists_date)
end_date = datetime.today().date()

StatementMeta(, 27e95eb9-9622-40eb-9543-3e616371ab29, 15, Finished, Available, Finished)

In [14]:
date_df = spark.sql(f"""
    SELECT explode(sequence(to_date('{start_date}'), to_date('{end_date}'), interval 1 day)) AS date
""")

date_dim = date_df.select(
    F.date_format('date', 'yyyyMMdd').cast(IntegerType()).alias('date_id'),
    F.col('date').cast(DateType()).alias('date'),
    F.year('date').alias('year'),
    F.month('date').alias('month'),
    F.weekofyear('date').alias('week'),
    F.dayofmonth('date').alias('day'),
    F.date_format('date', 'MMMM').alias('month_name')  # Add month name
)

date_dim.createOrReplaceTempView("date_dim")


StatementMeta(, 27e95eb9-9622-40eb-9543-3e616371ab29, 16, Finished, Available, Finished)

In [15]:
date_dim.write.mode('overwrite').option("overwriteSchema", "true").format("delta").saveAsTable("date_dim")

StatementMeta(, 27e95eb9-9622-40eb-9543-3e616371ab29, 17, Finished, Available, Finished)